In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_403079/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_39_0,0.862801,0.226548,0.009360,0.932703,0.023798,0.459155,0.693964,0.009257,0.059557,0.154265,0.677011,0.155637,137.476327,315.329227,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_38_9,0.862781,0.226598,0.009284,0.858173,0.023801,0.459125,0.694017,0.027942,0.059563,0.154276,0.676965,0.155648,137.476040,315.328939,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_38_8,0.862638,0.227513,0.011631,0.859191,0.023826,0.458582,0.692373,0.027741,0.059662,0.154357,0.676628,0.155729,137.473955,315.326854,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_38_6,0.861003,0.226880,0.032646,0.863022,0.024110,0.458958,0.677652,0.026986,0.060387,0.155273,0.672777,0.156653,137.450280,315.303179,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_38_7,0.860835,0.222902,0.025392,0.859392,0.024139,0.461319,0.682733,0.027701,0.060577,0.155367,0.672382,0.156748,137.447866,315.300765,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,model_1_17_9,-0.298537,-1.135514,-1.351854,-1.940174,0.225237,1.267733,1.647523,1.027833,0.096776,0.474592,-2.056972,0.478810,132.981203,310.834102,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
436,model_1_17_7,-0.303017,-1.066321,-0.786446,-1.934866,0.226014,1.226658,1.251443,1.025977,0.089249,0.475410,-2.067519,0.479635,132.974315,310.827214,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
437,model_1_17_4,-0.338872,-1.184404,-1.944273,-1.900174,0.232233,1.296757,2.062524,1.013850,0.093649,0.481906,-2.151928,0.486190,132.920025,310.772924,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
438,model_1_17_5,-0.370316,-1.178739,-1.884328,-2.013113,0.237688,1.293393,2.020531,1.053331,0.094119,0.487532,-2.225952,0.491866,132.873597,310.726496,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
